In [ ]:
from pathlib import Path
import json

In [ ]:
import pandas as pd

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use("grayscale")

# Results

In [ ]:
files = list()
for file in Path("results").glob("*.json"):
    with open(file, "r") as f:
        files.append(json.load(f))

df = pd.json_normalize(files)

In [ ]:
df["acc_diff"] = (df["acc_test"] - df["acc_train"]).abs()

In [ ]:
df["dataset"].unique()

In [ ]:
df.columns

In [ ]:
sns.lmplot(
    data=df, x="alpha", y="gg_border_perc", col="dataset",
    col_wrap=4, palette="muted", ci=None,
    height=3, scatter_kws={"s": 10, "alpha": .3},
)
plt.xscale("log")

In [ ]:
metrics = [
    "silhouette_score", "sil_neg_samples_score",
    "calinski_harabasz_score", "davies_bouldin_score",
    "gg_neigh_index", "gg_border_perc",
]

In [ ]:
aux = list()
for cdt in df["dataset"].unique():
    aux.append(
        df[df["dataset"] == cdt]
        .corr(numeric_only=True)
        [["alpha"]]
        .loc[metrics]
        .reset_index()
        .rename(columns={"index": "metrics"})
        .assign(dataset=cdt)
    )
corrs = pd.concat(aux)

In [ ]:
corrs.pivot(columns="dataset", index="metrics", values="alpha")

In [ ]:
df.corr(numeric_only=True)[["alpha"]].loc[metrics]

---